**2) 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)**

3. LDA의 수행하기

  1. 사용자는 알고리즘에게 토픽의 개수 k를 알려줍니다.
  2. 모든 단어를 k개 중 하나의 토픽에 할당합니다.
  3. 이제 모든 문서의 모든 단어에 대해서 아래의 사항을 반복 진행합니다. (iterative)  

5. 실습을 통한 이해

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers','footers','quotes'))
documents = dataset.data
print('뉴스그룹 데이터 개수 -------------------------') 
print(len(documents))
print()
print('첫번재 훈련용 데이터 -------------------------') 
print(documents[1])
print()
print('target_names ---------------------------------') 
print(dataset.target_names)
print()

In [ ]:
 import nltk
 nltk.download('stopwords')

# 1. 텍스트 전처리
news_df = pd.DataFrame({'document': documents})

# 특수문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]"," ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

print('원문 -----------------------------------------') 
print(news_df['document'][1])
print()
print('전처리 ---------------------------------------') 
print(news_df['clean_doc'][1])
print()

from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK 로 부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
print('불용어 제거 전 -------------------------------') 
print(tokenized_doc[1])
print()

tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words]) # 불용어 제거
print('불용어 제거 후 -------------------------------') 
print(tokenized_doc[1])
print()


1) 정수 인코딩과 단어 집합 만들기

In [ ]:
print(tokenized_doc[:5])
print()

from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
print(len(dictionary))
print()

corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력, 첫번째 문서의 인덱스는 0
print()

print(dictionary[66])
print()

2) LDA 모델 훈련시키기

In [ ]:
import gensim

NUM_TOPICS = 20 #20개의 토픽, k=20

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

print()
print(ldamodel.print_topics())

3) LDA 시각화 하기

In [ ]:
!pip3 install pyLDAvis

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

4) 문서 별 토픽 분포 보기

In [ ]:
for i, topic_list in enumerate(ldamodel[corpus]):
  if i == 5:
    break
  print(i, '번째 문서의 topic 비율은', topic_list)

In [ ]:
 def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # Ex) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호를 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호','가장 비중이 높은 토픽','가장 높은 토픽의 비중','각 토픽의 비중']
print(len(topictable))
print(topictable[:10])

http://s-space.snu.ac.kr/bitstream/10371/95582/1/22_1_%EB%82%A8%EC%B6%98%ED%98%B8.pdf
https://bab2min.tistory.com/568
https://annalyzin.wordpress.com/2015/06/21/laymans-explanation-of-topic-modeling-with-lda-2/
https://towardsdatascience.com/latent-dirichlet-allocation-15800c852699
https://radimrehurek.com/gensim/wiki.html#latent-dirichlet-allocation
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

모델 저장 및 로드 하기 : https://stackabuse.com/python-for-nlp-working-with-the-gensim-library-part-2/

전반적으로 참고하기 좋은 글 : https://shichaoji.com/2017/04/25/topicmodeling/

동영상 강의 : https://blog.naver.com/chunjein/220946362463

뉴스를 가지고 할 수 있는 다양한 일들 : https://www.slideshare.net/koorukuroo/20160813-pycon2016apac